<a href="https://colab.research.google.com/github/Reustlin/education_AutoML/blob/main/%D0%9A_%22YOLOv7ONNXandTRT_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade setuptools pip --user
!pip install onnx
!pip install onnxruntime
#!pip install --ignore-installed PyYAML
#!pip install Pillow

!pip install protobuf<4.21.3
!pip install onnxruntime-gpu
!pip install onnx>=1.9.0
!pip install onnx-simplifier>=0.3.6 --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.1/805.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


  Obtaining dependency information for onnx from https://files.pythonhosted.org/packages/47/d4/f2d212558245e252b936247666c3f5981e6dba62ec470ff8be3df3389364/onnx-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 67.8 MB/s eta 0:00:00
  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/2f/e2/ced4e64433097cb14425098ce3c6200b83d226005e8c23ba5bac44c89ab9/onnxruntime-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 23.8 MB/s eta 0:00:00
/bin/bash: line 1: 4.21.3: No such file or directory
  Obtaining dependency information for onnxruntime-gpu from https://files.pythonhosted.org/packages/a9/c6/855a9879e19dc44342a3a2b97c4258880ff841f

In [ ]:
import sys
import torch
print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0) 
Pytorch version: 2.0.1+cu118 


In [ ]:
!nvidia-smi

Wed Aug 30 07:53:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Total 1191 (delta 0), reused 0 (delta 0), pack-reused 1191
Receiving objects: 100% (1191/1191), 74.22 MiB | 19.34 MiB/s, done.
Resolving deltas: 100% (515/515), done.
/content/yolov7
cfg	detect.py  hubconf.py  models	  requirements.txt  tools	  utils
data	export.py  inference   paper	  scripts	    train_aux.py
deploy	figure	   LICENSE.md  README.md  test.py	    train.py


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

# clone YOLOv7 repository
%cd {HOME}
!git clone https://github.com/WongKinYiu/yolov7

# navigate to yolov7 directory and checkout u7 branch of YOLOv7 - this is hash of lates commit from u7 branch as of 12/21/2022
%cd {HOME}/yolov7
!git checkout 44f30af0daccb1a3baecc5d80eae22948516c579

/content/yolov7
/content/yolov7
Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Total 1191 (delta 0), reused 0 (delta 0), pack-reused 1191
Receiving objects: 100% (1191/1191), 74.23 MiB | 1.06 MiB/s, done.
Resolving deltas: 100% (513/513), done.
/content/yolov7/yolov7
Note: switching to '44f30af0daccb1a3baecc5d80eae22948516c579'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 44f30af u7 readme


In [ ]:
%cd /content/yolov7/seg
!python export.py --weights bestseg.pt --include torchscript onnx --img 256 256

/content/yolov7/seg
export: data=data/coco128.yaml, weights=['bestseg.pt'], imgsz=[256, 256], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 2023-8-30 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
yolov7-seg summary: 325 layers, 37842476 parameters, 0 gradients

PyTorch: starting from bestseg.pt with output shape (1, 4032, 38) (72.6 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 5.3s, saved as bestseg.torchscript (144.8 MB)

ONNX: starting export with onnx 1.14.1...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: 

In [ ]:
# export ONNX for ONNX inference
%cd /content/yolov7/
!python export.py --weights ./bestsushi.pt \
        --grid --end2end --simplify \
        --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 \
        --img-size 352 640 --max-wh 0 # For onnxruntime, you need to specify this value as an integer, when it is 0 it means agnostic NMS,
                     # otherwise it is non-agnostic NMS

/content/yolov7
Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='./bestsushi.pt', img_size=[352, 640], batch_size=1, dynamic=False, dynamic_batch=False, grid=True, end2end=True, max_wh=0, topk_all=100, iou_thres=0.65, conf_thres=0.35, device='cpu', simplify=True, include_nms=False, fp16=False, int8=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CPU

Fusing layers... 
IDetect.fuse
Model Summary: 208 layers, 6007596 parameters, 0 gradients
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting TorchScript export with torch 2.0.1+cu118...
/content/yolov7/models/yolo.py:150: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record t